## Unavailable, but inactive, English word usernames

This is not to be abused. Please do not steal usernames from people.

In [1]:
from dateutil.relativedelta import relativedelta
from typing import List, Tuple, Dict
from datetime import datetime
from progress.bar import Bar

import requests
import os

In [2]:
github_user_url = "https://api.github.com/users/%s"
auth_token = "token " + os.environ['GITHUB_ACCESS_TOKEN']

I'm getting my list from `https://github.com/mahsu/IndexingExercise/blob/master/5000-words.txt`, but you can change this to your own list or w/e.

In [3]:
def get_words(filename: str) -> List:
    with open(filename) as file:
        return list(file.read().split())

In [4]:
data = get_words('5000-words.txt')
print('Number of words: %d' % (len(data)))

Number of words: 5008


Because there's a lot of data from GitHub's response, it's better to just parse out the two fields that we are interested in and then return them as a dict.

In [5]:
def get_github_data(username: str) -> List:
    result = {}
    headers = {"Authorization": auth_token}
    with requests.session() as session:
        json = session.get(
            github_user_url % username, 
            headers=headers
        ).json()
        
        # message field only exists if user doesn't exist
        if 'message' in json:
            return {}
        result['updated_at'] = json['updated_at']
    return result

Now we need a way to validate the dates, this is obviously not perfect, because it doesn't account for private changes to the account.

In [6]:
def has_recent_activity(username: str) -> bool:
    values = get_github_data(username)
    # check if `values` is empty, meaning the user doesn't exist
    if len(values) == 0:
        return False
    # the date format is: %Y-%m-%dT%H:%M:%SZ
    format = "%Y-%m-%dT%H:%M:%SZ"
    updated_at = datetime.strptime(values['updated_at'], format).date()
    diff = relativedelta(datetime.now(), updated_at).years
    # we need to check the inactivity
    # randomly chosen amount of years of inactivity: 5
    if diff >= 5:
        return False
    return True

In [7]:
user = get_github_data('hum')
print(user)

{'updated_at': '2021-03-24T15:33:09Z'}


In [8]:
print(has_recent_activity('hum'))

True


Let's test this on an old and inactive account. `https://github.com/bruh`

In [9]:
print(has_recent_activity('totally-doesnt-exist'))

False


Now we can run it on our list of words `data`. A sample result is included with this in the github repo.

In [ ]:
with open('inactive_accounts.txt', 'a', encoding='utf-8') as file:
    for i in range(len(data)):
        username = data[i]
        
        if not has_recent_activity(username):
            file.write(username + "\n")